In [364]:
import causalnex
import networkx as nx
from causalnex.structure.dynotears import from_pandas_dynamic
from causalnex.structure.pytorch.notears import from_pandas
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch
import warnings
warnings.filterwarnings('ignore')

In [365]:
df_attack_train = pd.read_csv('../SWaT_Dataset/2015/cleaned_csv/attack_data_trainV2.csv',index_col=0).reset_index(drop=True)
df_attack_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,2015-12-25 13:15:00,2.667264,495.8409,2,2,1,261.0549,8.388426,331.1971,2.442956,...,0.306889,2,1,251.3939,1.762086,190.0795,0.000128,1,1,1
1,2015-12-25 13:15:01,2.622746,623.9226,2,1,1,260.2858,8.473982,318.5594,0.000000,...,0.306825,2,1,252.1148,1.537821,190.4480,0.000128,1,1,1
2,2015-12-25 13:15:02,2.537552,491.6801,2,2,1,263.2018,8.346770,332.2481,2.449108,...,0.306761,2,1,248.5100,1.105309,187.9326,0.000000,1,1,1
3,2015-12-25 13:15:03,2.493034,779.9130,2,1,1,263.9387,8.428160,319.2002,0.000000,...,0.306761,2,1,250.4967,1.105309,189.4546,0.000000,1,1,1
4,2015-12-25 13:15:04,0.932326,808.6853,1,1,1,260.4140,8.449949,319.3027,0.000000,...,0.306505,2,1,250.1602,1.169384,188.8618,0.000064,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14677,2015-12-25 17:19:37,0.000000,547.2620,1,2,1,264.4514,8.356383,332.4532,2.444751,...,0.304391,2,1,247.1962,1.169384,186.4426,0.000000,1,1,1
14678,2015-12-25 17:19:38,2.532428,520.1776,2,2,1,258.2992,8.375288,331.9918,2.441547,...,0.306569,2,1,247.9813,0.961138,186.8431,0.000000,1,1,1
14679,2015-12-25 17:19:39,2.659258,611.0084,2,1,1,260.4140,8.469175,318.6363,0.000000,...,0.306825,2,1,251.6182,1.537821,190.0474,0.000000,1,1,1
14680,2015-12-25 17:19:40,2.513211,510.3643,2,2,1,258.7157,8.392271,333.1197,2.442444,...,0.308426,2,1,253.1242,1.281517,191.2170,0.000064,1,1,1


In [366]:
df_normal_train = pd.read_csv('../SWaT_Dataset/2015/cleaned_csv/normal_data_trainV2.csv',index_col=0).reset_index(drop=True)
df_normal_train

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
0,2015-12-22 16:30:00,0.000000,124.3135,1,1,1,251.9226,8.313446,312.7916,0.000000,...,0.000000,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1
1,2015-12-22 16:30:01,0.000000,124.3920,1,1,1,251.9226,8.313446,312.7916,0.000000,...,0.000000,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1
2,2015-12-22 16:30:02,0.000000,124.4705,1,1,1,251.9226,8.313446,312.7916,0.000000,...,0.000000,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1
3,2015-12-22 16:30:03,0.000000,124.6668,1,1,1,251.9226,8.313446,312.7916,0.000000,...,0.000000,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1
4,2015-12-22 16:30:04,0.000000,124.5098,1,1,1,251.9226,8.313446,312.7916,0.000000,...,0.000000,1,1,9.100231,0.000000,3.3485,0.000256,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247495,2015-12-25 13:14:55,2.534029,506.0858,2,2,1,266.3419,8.353820,327.7108,2.448467,...,0.308362,2,1,250.865200,1.137347,189.9673,0.000000,1,1,1
247496,2015-12-25 13:14:56,2.522499,505.8896,2,2,1,266.3419,8.353820,327.7108,2.448467,...,0.308362,2,1,250.865200,1.137347,190.0795,0.000000,1,1,1
247497,2015-12-25 13:14:57,2.510329,505.6933,2,2,1,266.3419,8.353820,327.7878,2.448467,...,0.308362,2,1,250.865200,1.137347,190.0795,0.000000,1,1,1
247498,2015-12-25 13:14:58,2.494635,505.6148,2,2,1,266.3419,8.353820,327.7878,2.448595,...,0.308362,2,1,250.865200,1.137347,190.1596,0.000000,1,1,1


In [429]:
df_test = pd.read_csv('../SWaT_Dataset/2015/cleaned_csv/test_dataV2.csv',index_col=0).reset_index(drop=True)
df_test

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,2015-12-25 17:30:00,2.490472,515.3887,2,2,1,240.3871,8.415983,333.9913,2.446801,...,2,1,249.6155,0.929100,188.3972,0.000000,1,1,1,Normal
1,2015-12-25 17:30:01,2.476059,515.2317,2,2,1,240.3871,8.415983,333.9913,2.446801,...,2,1,249.6155,0.929100,188.3972,0.000000,1,1,1,Normal
2,2015-12-25 17:30:02,2.457483,515.2709,2,2,1,240.3871,8.415983,333.9913,2.445519,...,2,1,249.6155,0.929100,188.2530,0.000000,1,1,1,Normal
3,2015-12-25 17:30:03,2.443071,515.6242,2,2,1,240.3871,8.414381,333.9913,2.443982,...,2,1,249.6155,0.929100,188.2530,0.000000,1,1,1,Normal
4,2015-12-25 17:30:04,2.425776,515.3102,2,2,1,240.3871,8.414381,333.9913,2.443982,...,2,1,249.6155,0.929100,188.2530,0.000000,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314995,2015-12-29 08:59:55,2.500080,509.7363,2,2,1,264.7078,8.352859,331.5047,2.428475,...,2,1,247.9172,1.121328,187.1155,0.000000,1,1,1,Normal
314996,2015-12-29 08:59:56,2.479903,655.4818,2,1,1,265.8934,8.453794,316.3291,0.000000,...,2,1,251.3779,1.105309,190.1756,0.000064,1,1,1,Normal
314997,2015-12-29 08:59:57,2.500721,499.9231,2,2,1,260.1577,8.402525,333.1710,2.442444,...,2,1,251.9546,1.505783,190.4800,0.000128,1,1,1,Normal
314998,2015-12-29 08:59:58,2.609935,520.5701,2,2,1,270.9562,8.338759,328.3773,2.429501,...,2,1,250.0961,1.169384,189.2944,0.000000,1,1,1,Normal


In [430]:
df_test['Normal/Attack'] = df_test['Normal/Attack'].replace({'Normal':0 , 'Attack':1})

In [431]:
df_attack_train['Timestamp'] = pd.to_datetime(df_attack_train['Timestamp'])
df_normal_train['Timestamp'] = pd.to_datetime(df_normal_train['Timestamp'])
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])

In [370]:
df_attack_train['Timestamp'] = df_attack_train['Timestamp'].apply(lambda x:x.value)

In [371]:
struct_model_attack_dense = from_pandas_dynamic(df_attack_train, 4)

In [372]:
df_attack_nodes = pd.read_csv('../SWaT_Dataset/2015/cleaned_csv/attacks.csv').reset_index(drop=True)
attack_nodes = df_attack_nodes['Attack Point'].tolist()
attack_nodes

['MV101',
 'P102',
 'LIT101',
 'MV504',
 'AIT202',
 'LIT301',
 'DPIT301',
 'FIT401',
 'FIT401',
 'MV304',
 'Mv303',
 'LIT301',
 'MV303',
 'AIT504',
 'AIT504',
 'MV101',
 'UV401',
 'P602',
 'P203',
 'LIT401',
 'P101',
 'P302',
 'P302',
 'P201',
 'LIT101',
 'LIT401',
 'LIT301',
 'LIT101',
 'P101',
 'P101',
 'LIT101',
 'P501',
 'AIT402',
 'FIT401',
 'FIT401',
 'LIT301',
 'AIT502',
 'DIT301',
 'MV302',
 'P205',
 'P401',
 'FIT502']

In [373]:
attack_nodes_tl = []
for node in attack_nodes:
    for i in range(0,5):
        add_tl = ''
        add_tl = node+'_lag'+str(i)
        attack_nodes_tl.append(add_tl)

attack_nodes_tl


['MV101_lag0',
 'MV101_lag1',
 'MV101_lag2',
 'MV101_lag3',
 'MV101_lag4',
 'P102_lag0',
 'P102_lag1',
 'P102_lag2',
 'P102_lag3',
 'P102_lag4',
 'LIT101_lag0',
 'LIT101_lag1',
 'LIT101_lag2',
 'LIT101_lag3',
 'LIT101_lag4',
 'MV504_lag0',
 'MV504_lag1',
 'MV504_lag2',
 'MV504_lag3',
 'MV504_lag4',
 'AIT202_lag0',
 'AIT202_lag1',
 'AIT202_lag2',
 'AIT202_lag3',
 'AIT202_lag4',
 'LIT301_lag0',
 'LIT301_lag1',
 'LIT301_lag2',
 'LIT301_lag3',
 'LIT301_lag4',
 'DPIT301_lag0',
 'DPIT301_lag1',
 'DPIT301_lag2',
 'DPIT301_lag3',
 'DPIT301_lag4',
 'FIT401_lag0',
 'FIT401_lag1',
 'FIT401_lag2',
 'FIT401_lag3',
 'FIT401_lag4',
 'FIT401_lag0',
 'FIT401_lag1',
 'FIT401_lag2',
 'FIT401_lag3',
 'FIT401_lag4',
 'MV304_lag0',
 'MV304_lag1',
 'MV304_lag2',
 'MV304_lag3',
 'MV304_lag4',
 'Mv303_lag0',
 'Mv303_lag1',
 'Mv303_lag2',
 'Mv303_lag3',
 'Mv303_lag4',
 'LIT301_lag0',
 'LIT301_lag1',
 'LIT301_lag2',
 'LIT301_lag3',
 'LIT301_lag4',
 'MV303_lag0',
 'MV303_lag1',
 'MV303_lag2',
 'MV303_lag3',
 'MV3

In [374]:
struct_model_attack_sparse = struct_model_attack_dense.subgraph(attack_nodes_tl)

In [375]:
sparse_nodes = []
for node in struct_model_attack_dense.nodes:
    if node not in struct_model_attack_sparse:
        sparse_nodes.append(node)

len(sparse_nodes)

140

In [376]:
df_normal_train['Timestamp'] = df_normal_train['Timestamp'].apply(lambda x:x.value)

In [377]:
struct_model_normal_dense = from_pandas_dynamic(df_normal_train, 4)

In [378]:
import numpy as np

def create_laplacian_with_subgraph(adjacency_matrix, subgraph_nodes, subgraph_weight):
    for node in subgraph_nodes:
        adjacency_matrix[node, subgraph_nodes] *= subgraph_weight
    
    degree_matrix = np.diag(np.sum(adjacency_matrix, axis=1))
    
    laplacian_matrix = degree_matrix - adjacency_matrix
    
    return laplacian_matrix

In [379]:
attack_node_dict = {node: i for i, node in enumerate(struct_model_attack_dense.nodes)}
attack_node_dict

{'Timestamp_lag0': 0,
 'Timestamp_lag1': 1,
 'Timestamp_lag2': 2,
 'Timestamp_lag3': 3,
 'Timestamp_lag4': 4,
 'FIT101_lag0': 5,
 'FIT101_lag1': 6,
 'FIT101_lag2': 7,
 'FIT101_lag3': 8,
 'FIT101_lag4': 9,
 'LIT101_lag0': 10,
 'LIT101_lag1': 11,
 'LIT101_lag2': 12,
 'LIT101_lag3': 13,
 'LIT101_lag4': 14,
 'MV101_lag0': 15,
 'MV101_lag1': 16,
 'MV101_lag2': 17,
 'MV101_lag3': 18,
 'MV101_lag4': 19,
 'P101_lag0': 20,
 'P101_lag1': 21,
 'P101_lag2': 22,
 'P101_lag3': 23,
 'P101_lag4': 24,
 'P102_lag0': 25,
 'P102_lag1': 26,
 'P102_lag2': 27,
 'P102_lag3': 28,
 'P102_lag4': 29,
 'AIT201_lag0': 30,
 'AIT201_lag1': 31,
 'AIT201_lag2': 32,
 'AIT201_lag3': 33,
 'AIT201_lag4': 34,
 'AIT202_lag0': 35,
 'AIT202_lag1': 36,
 'AIT202_lag2': 37,
 'AIT202_lag3': 38,
 'AIT202_lag4': 39,
 'AIT203_lag0': 40,
 'AIT203_lag1': 41,
 'AIT203_lag2': 42,
 'AIT203_lag3': 43,
 'AIT203_lag4': 44,
 'FIT201_lag0': 45,
 'FIT201_lag1': 46,
 'FIT201_lag2': 47,
 'FIT201_lag3': 48,
 'FIT201_lag4': 49,
 'MV201_lag0': 50,
 

In [380]:
struct_model_attack_dense_adj = np.zeros((260,260))
for (a,b) in struct_model_attack_dense.edges:
    struct_model_attack_dense_adj[attack_node_dict[a], attack_node_dict[b]] = 1

struct_model_attack_dense_adj

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [381]:
subgraph_nodes = []
for subgraph_node in struct_model_attack_sparse.nodes:
    subgraph_nodes.append(attack_node_dict[subgraph_node])

subgraph_weight = 3

laplacian_with_subgraph_attack = create_laplacian_with_subgraph(struct_model_attack_dense_adj, subgraph_nodes, subgraph_weight)

print(laplacian_with_subgraph_attack)

[[51.  0.  0. ...  0.  0.  0.]
 [-1. 52.  0. ...  0.  0.  0.]
 [-1.  0. 52. ...  0.  0.  0.]
 ...
 [-1.  0.  0. ... 51.  0.  0.]
 [-1.  0.  0. ...  0. 51.  0.]
 [-1.  0.  0. ...  0.  0. 51.]]


In [382]:
normal_node_dict = {node: i for i, node in enumerate(struct_model_normal_dense.nodes)}
struct_model_normal_dense_adj = np.zeros((260,260))
for (a,b) in struct_model_normal_dense.edges:
    struct_model_normal_dense_adj[normal_node_dict[a], normal_node_dict[b]] = 1

struct_model_normal_dense_adj

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [383]:
normal_degree_matrix = np.diag(np.sum(struct_model_normal_dense_adj, axis=1))

laplacian_normal = normal_degree_matrix - struct_model_normal_dense_adj

laplacian_normal

array([[51.,  0.,  0., ...,  0.,  0.,  0.],
       [-1., 52.,  0., ...,  0.,  0.,  0.],
       [-1.,  0., 52., ...,  0.,  0.,  0.],
       ...,
       [-1.,  0.,  0., ..., 51.,  0.,  0.],
       [-1.,  0.,  0., ...,  0., 51.,  0.],
       [-1.,  0.,  0., ...,  0.,  0., 51.]])

In [384]:
def getEdgeIndex(graph):

    row_ind, col_ind = graph.nonzero()

    edge_index = torch.tensor(np.vstack([col_ind, row_ind]).T)

    return edge_index

In [423]:
import torch
from torch_geometric.nn import GCNConv, Linear

class GNN_Classifier(torch.nn.Module):
  def __init__(self, features_dim):
    super(GNN_Classifier, self).__init__()

    self.conv1 = GCNConv(features_dim, 16)  
    self.conv2 = GCNConv(16, 8)  
    self.conv3 = Linear(8, 1) 

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index).relu()
    x = self.conv2(x, edge_index).relu()
    x = torch.mean(x, dim=1)
    x = self.conv3(x).relu()
    return torch.sigmoid(x)
  
laplacian_1 = torch.tensor(laplacian_with_subgraph_attack) 
laplacian_2 = torch.tensor(laplacian_normal)

edge_index_adj_attack = getEdgeIndex(struct_model_attack_dense_adj)
edge_index_adj_normal = getEdgeIndex(struct_model_normal_dense_adj)

data_list = [
    {'x': laplacian_1.double(), 'y': torch.tensor([1]), 'edge_index': torch.tensor(edge_index_adj_attack[:13070])},  # Label 1
    {'x': laplacian_2.double(), 'y': torch.tensor([0]), 'edge_index': torch.tensor(edge_index_adj_normal)}   # Label 2
]


In [424]:
from torch_geometric.data import DataLoader

dataset = DataLoader(data_list, batch_size=2)

model = GNN_Classifier(laplacian_1.shape[0]).double()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

In [425]:
for data in dataset:
    print (data)

{'x': tensor([[[51.,  0.,  0.,  ...,  0.,  0.,  0.],
         [-1., 52.,  0.,  ...,  0.,  0.,  0.],
         [-1.,  0., 52.,  ...,  0.,  0.,  0.],
         ...,
         [-1.,  0.,  0.,  ..., 51.,  0.,  0.],
         [-1.,  0.,  0.,  ...,  0., 51.,  0.],
         [-1.,  0.,  0.,  ...,  0.,  0., 51.]],

        [[51.,  0.,  0.,  ...,  0.,  0.,  0.],
         [-1., 52.,  0.,  ...,  0.,  0.,  0.],
         [-1.,  0., 52.,  ...,  0.,  0.,  0.],
         ...,
         [-1.,  0.,  0.,  ..., 51.,  0.,  0.],
         [-1.,  0.,  0.,  ...,  0., 51.,  0.],
         [-1.,  0.,  0.,  ...,  0.,  0., 51.]]], dtype=torch.float64), 'y': tensor([[1],
        [0]]), 'edge_index': tensor([[[  5,   0],
         [ 10,   0],
         [ 15,   0],
         ...,
         [ 90, 259],
         [ 95, 259],
         [100, 259]],

        [[  5,   0],
         [ 10,   0],
         [ 15,   0],
         ...,
         [245, 259],
         [250, 259],
         [255, 259]]])}


In [426]:
model.train()
for epoch in range(58):  
  for data in dataset:
    x, y, edge_index = data['x'], data['y'], data['edge_index']
    optimizer.zero_grad()
    pred = model(x, edge_index) 
    loss = criterion(pred, y.double())
    loss.backward()
    optimizer.step()

  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 58, loss.item()))

  #Ass dropout layer

Epoch [1/58], Loss: 0.7113
Epoch [2/58], Loss: 0.6725
Epoch [3/58], Loss: 0.6490
Epoch [4/58], Loss: 0.6201
Epoch [5/58], Loss: 0.5850
Epoch [6/58], Loss: 0.5580
Epoch [7/58], Loss: 0.5351
Epoch [8/58], Loss: 0.5196
Epoch [9/58], Loss: 0.4917
Epoch [10/58], Loss: 0.4679
Epoch [11/58], Loss: 0.4463
Epoch [12/58], Loss: 0.4389
Epoch [13/58], Loss: 0.4133
Epoch [14/58], Loss: 0.4167
Epoch [15/58], Loss: 0.4152
Epoch [16/58], Loss: 0.4015
Epoch [17/58], Loss: 0.3837
Epoch [18/58], Loss: 0.3688
Epoch [19/58], Loss: 0.4140
Epoch [20/58], Loss: 0.3627
Epoch [21/58], Loss: 0.3698
Epoch [22/58], Loss: 0.3830
Epoch [23/58], Loss: 0.3945
Epoch [24/58], Loss: 0.3971
Epoch [25/58], Loss: 0.3891
Epoch [26/58], Loss: 0.3762
Epoch [27/58], Loss: 0.3648
Epoch [28/58], Loss: 0.3572
Epoch [29/58], Loss: 0.3526
Epoch [30/58], Loss: 0.3501
Epoch [31/58], Loss: 0.3486
Epoch [32/58], Loss: 0.4006
Epoch [33/58], Loss: 0.3484
Epoch [34/58], Loss: 0.3491
Epoch [35/58], Loss: 0.3499
Epoch [36/58], Loss: 0.3510
E

In [432]:
y_true = df_test['Normal/Attack']

In [433]:
df_test.drop(columns='Normal/Attack', inplace=True)

In [434]:
any(y_true[:900])==1

False

In [435]:
test_data_list = []
for i in range(0, len(df_test),900):
    df_struct_test = df_test[i:i+900]
    y = 0
    if (any(y_true[i:i+900])==1):
        y = 1
    df_struct_test['Timestamp'] = df_struct_test['Timestamp'].apply(lambda x: x.value)
    struct_model_test = from_pandas_dynamic(df_struct_test, 4)

    test_node_dict = {node: i for i, node in enumerate(struct_model_test.nodes)}
    struct_model_test_adj = np.zeros((260,260))
    for (a,b) in struct_model_test.edges:
        struct_model_test_adj[test_node_dict[a], test_node_dict[b]] = 1

    test_degree_matrix = np.diag(np.sum(struct_model_test_adj, axis=1))

    laplacian_test = torch.tensor(test_degree_matrix - struct_model_test_adj)

    test_edgeIndex = torch.tensor(getEdgeIndex(struct_model_test_adj))


    test_data_list.append({'x': laplacian_test.double(), 'y': torch.tensor([y]), 'edge_index': torch.tensor(test_edgeIndex)[:8000]})


In [436]:
test_dataset = DataLoader(test_data_list, batch_size=2)

test_dataset

In [437]:
for test in test_dataset:
    print (test)

{'x': tensor([[[51.,  0.,  0.,  ...,  0.,  0.,  0.],
         [-1., 52.,  0.,  ...,  0.,  0.,  0.],
         [-1.,  0., 52.,  ...,  0.,  0.,  0.],
         ...,
         [-1.,  0.,  0.,  ..., 51.,  0.,  0.],
         [-1.,  0.,  0.,  ...,  0., 51.,  0.],
         [-1.,  0.,  0.,  ...,  0.,  0., 51.]],

        [[51.,  0.,  0.,  ...,  0.,  0.,  0.],
         [-1., 52.,  0.,  ...,  0.,  0.,  0.],
         [-1.,  0., 52.,  ...,  0.,  0.,  0.],
         ...,
         [-1.,  0.,  0.,  ..., 51.,  0.,  0.],
         [-1.,  0.,  0.,  ...,  0., 51.,  0.],
         [-1.,  0.,  0.,  ...,  0.,  0., 51.]]], dtype=torch.float64), 'y': tensor([[0],
        [0]]), 'edge_index': tensor([[[  5,   0],
         [ 10,   0],
         [ 15,   0],
         ...,
         [ 85, 157],
         [ 90, 157],
         [ 95, 157]],

        [[  5,   0],
         [ 10,   0],
         [ 15,   0],
         ...,
         [160, 157],
         [165, 157],
         [170, 157]]])}
{'x': tensor([[[50.,  0.,  0.,  ...,  0.,  0

In [438]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, roc_auc_score
model.eval()
total_correct = 0
total_samples = 0
y_pred = []
with torch.no_grad():
    for test_data in test_dataset:
        x_test, y_test, edge_index_test = test_data['x'], test_data['y'], test_data['edge_index']
        pred_test = model(x_test, edge_index_test)
        pred_label = (pred_test>0.5).float()

        y_pred.append(pred_label)
        correct = pred_label.eq(y_test.view_as(pred_label)).sum().item()
        total_correct += correct
        total_samples += y_test.size(0)

accuracy = total_correct / total_samples
print("Test Accuracy: {:.4f}".format(accuracy))



Test Accuracy: 0.8914


In [439]:
y_pred_array = []
for ten in y_pred:
    numpy_array = ten.numpy()
    ten_list = numpy_array.squeeze().tolist()
    y_pred_array.append(ten_list)

y_pred_array


[[0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [1.0, 1.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],

In [440]:
y_pred_df = pd.DataFrame([item for sublist in y_pred_array for item in sublist])

y_pred_df = y_pred_df[0].astype(int)
y_pred_df

0      0
1      0
2      0
3      0
4      0
      ..
345    0
346    0
347    0
348    0
349    0
Name: 0, Length: 350, dtype: int32

In [441]:
y_true_df = []
for i in range(0, len(y_true), 900):
    # print(sum(y_true[i:i+900]))
    if (sum(y_true[i:i+900])>0):
        y_true_df.append(1)
    else:
        y_true_df.append(0)


y_true_df = pd.DataFrame({'0': y_true_df})
y_true_df = y_true_df['0']

In [442]:
# Calculate evaluation metrics
precision = precision_score(y_true_df, y_pred_df)
recall = recall_score(y_true_df, y_pred_df)
f1 = f1_score(y_true_df, y_pred_df)
accuracy = accuracy_score(y_true_df, y_pred_df)
roc_auc = roc_auc_score(y_true_df, y_pred_df)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Accuracy:", accuracy)
print("ROC AUC score:", roc_auc)

Precision: 0.7049180327868853
Recall: 0.6825396825396826
F1-score: 0.6935483870967742
Accuracy: 0.8914285714285715
ROC AUC score: 0.8099109562524196


In [419]:
y_true_df

0      0
1      0
2      0
3      0
4      0
      ..
345    0
346    0
347    0
348    0
349    0
Name: 0, Length: 350, dtype: int64

In [23]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from scipy.sparse import csr_matrix

class GAT(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes, num_heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_feats, hidden_size, num_heads)
        self.conv2 = GATConv(hidden_size * num_heads, num_classes, 1)

    def forward(self, g, features):
        x = F.elu(self.conv1(g, features))
        x = self.conv2(g, x).mean(1)
        return x


# Load your Laplacian matrices and graph data
# Assuming you have the Laplacian matrices as L1 and L2
# Assuming you have the graph data as graph_data

# Assuming you have labels for the two graphs as labels1 and labels2
labels1 = torch.tensor([1])  # Label for the first graph
labels2 = torch.tensor([0])  # Label for the second graph

# Convert Laplacian matrices and graph data to DGLGraph
g1 = dgl.from_scipy(csr_matrix(laplacian_with_subgraph_attack))
g2 = dgl.from_scipy(csr_matrix(laplacian_normal))

# Define model parameters
in_feats = g1.number_of_nodes()  # Assuming the input feature dimension is equal to the number of nodes
hidden_size = 16  # Hidden layer size
num_classes = 2  # Number of output classes
num_heads = 2  # Number of attention heads

# Create GAT model
model = GAT(in_feats, hidden_size, num_classes, num_heads)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    # Forward pass for the first graph
    logits1 = model(g1, g1.ndata['feat'].to(device))
    # Forward pass for the second graph
    logits2 = model(g2, g2.ndata['feat'].to(device))
    # Calculate loss for both graphs
    loss = F.cross_entropy(torch.cat([logits1, logits2]), torch.cat([labels1, labels2]).to(device))
    loss.backward()
    optimizer.step()

    # # Evaluation on test data
    # model.eval()
    # # Assuming you have test graph data as test_graph_data






    # test_graph = dgl.from_scipy(test_graph_data)
    # with torch.no_grad():
    #     logits_test = model(test_graph, test_graph.ndata['feat'].to(device))
    #     predicted_labels = torch.argmax(logits_test, dim=1)

    # print("Predicted labels for test graph:", predicted_labels)


KeyError: 'feat'

In [27]:
csr_matrix(laplacian_with_subgraph_attack)

(260, 260)

In [29]:
g1.adj

<bound method DGLGraph.adj of Graph(num_nodes=260, num_edges=13360,
      ndata_schemes={}
      edata_schemes={})>

In [ ]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score, roc_auc_score
y_pred = []
for i in range(0, len(df_test),900):
    df_struct_test = df_test[i:i+900]
    df_struct_test['Timestamp'] = df_struct_test['Timestamp'].apply(lambda x: x.value)
    struct_model_test = from_pandas_dynamic(df_struct_test, 4)

